In [1]:
#1 load the data
from langchain_community.document_loaders.csv_loader import CSVLoader

#A document loader
loader = CSVLoader('laptops.csv', encoding='ISO-8859-1')

#Load the document
data = loader.load()
print(data[0])

page_content='Manufacturer: Apple
Model Name: MacBook Pro
Category: Ultrabook
Screen Size: 13.3"
Screen: IPS Panel Retina Display 2560x1600
CPU: Intel Core i5 2.3GHz
RAM: 8GB
Storage: 128GB SSD
GPU: Intel Iris Plus Graphics 640
Operating System: macOS
Operating System Version: 
Weight: 1.37kg
Price (Euros): 1339,69' metadata={'source': 'laptops.csv', 'row': 0}


In [2]:
#split the data into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(separators=['\n\n', '\n', ': '],chunk_size=500,chunk_overlap=50)
docs = splitter.split_documents(data)

print(docs[4])

page_content='Manufacturer: Apple
Model Name: MacBook Pro
Category: Ultrabook
Screen Size: 13.3"
Screen: IPS Panel Retina Display 2560x1600
CPU: Intel Core i5 3.1GHz
RAM: 8GB
Storage: 256GB SSD
GPU: Intel Iris Plus Graphics 650
Operating System: macOS
Operating System Version: 
Weight: 1.37kg
Price (Euros): 1803,60' metadata={'source': 'laptops.csv', 'row': 4}


In [3]:
#set the embedding
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name
)


C:\Users\HP\AppData\Local\Temp\ipykernel_12820\2575904129.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\HP\Documents\Stage - Cetic\Code\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\HP\Documents\Stage - Cetic\Code\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This

In [4]:
#set up Chroma vector store and retriever
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [5]:
#set the llm
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    raise ValueError("API key not set")

#load the model 
llm_groq = ChatGroq(
    groq_api_key=api_key,
    model_name='llama3-8b-8192'
)

In [6]:
# history aware retriever setup
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = (
    "Given a chat history where a user is looking for a laptop based on specific requirements, "
    "formulate a standalone question that clearly identifies the user's laptop needs without referring "
    "to previous chat messages. The goal is to ensure the question can be understood and processed "
    "independently. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(llm_groq, retriever, contextualize_q_prompt)


In [7]:
# question-answer chain

from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an assistant specialized in helping users find the best laptop from the available data based on their requirements. "
    "Use the provided laptop specifications and user preferences to recommend the most suitable options. "
    "do not use sentences like this Based on the provided data instead you can say in our store thats what we have"
    "If the user's requirements cannot be fully met, suggest the closest alternatives available. "
    "Keep your answers concise and focused on the laptop's key features such as price, performance, battery life, and portability. "
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm_groq, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [9]:
conversational_rag_chain.invoke(
    {"input": "What is The best laptop for design ?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  
)["answer"]

'In our store, I\'d recommend the Dell Inspiron 5379 2 in 1 Convertible for design purposes. Its 13.3" Full HD touchscreen display and Intel Core i5 processor provide a great balance of portability and performance for creative tasks. The 8GB RAM and 1TB HDD storage ensure smooth multitasking and ample storage for your files.\n\nHowever, keep in mind that the Intel UHD Graphics 620 might not be as powerful as a dedicated graphics card, which might be a limitation for more demanding design tasks. But for general graphic design, video editing, and other creative work, this laptop should suffice.\n\nIf you\'d like a larger screen, the Dell Inspiron 5579 2 in 1 Convertible is also a great option, but it\'s slightly heavier and more expensive.'